**Load necessary packages**

In [5]:
import tell
import im3components 

**Install package data**

NOTE: The package data will require approximately 1.4 GB of storage.

Set the local directory where you would like to store the package data and run the function below:

In [6]:
# Change to your local directory where you would like to store the data
data_dir = 'C:/Users/mcgr323/projects/tell_valid/tell/tell_raw_data'

# Download the raw data from the Zenodo package
tell.install_package_data(data_dir = data_dir)

AttributeError: module 'tell' has no attribute 'install_package_data'

**1. Data processing for TELL**

In the next few code blocks we will load and manipulate the nescessary data for the <b>tell</b>  package. This consists of hourly load, population and meterology for the CONUS, which will be loaded in from raw data sources, manipulated and then compiled together to use as input for the MLP model. Please follow the steps below to produce the hourly input data, if you have already finished this step you can proceed to **2. Model training and prediction**
    
**1.1 Hourly load**

Here we load in the raw EIA 930 hourly load profiles for all Balancing Authorities (BAs), subset for the wanted columns only and then output the hourly load as csvs to be compiled later with population, and meteorlogy to be fed to the MLP model downstream for predict future load. 

In [2]:
# Hourly load data (EIA 930):

# Set the data input and output directories:
EIA_930_input_dir = 'C:/Users/mcgr323/projects/tell_valid/tell/tell_raw_data'
EIA_930_output_dir = 'C:/Users/mcgr323/projects/tell/ba_hourly_inputs/ba_Hourly_Load'

# Process the hourly load data
tell.process_eia_930(EIA_930_input_dir, EIA_930_output_dir)

NameError: name 'tell' is not defined

**1.2 Population data**

For this data processing step we will load in the annual population by FIPS code, merge by FIPS code to get the correspondng BA number, sum by  year and BA number and then interpolate the annual population to hourly population in order to feed it to the MLP model downstream. 

In [3]:
# County population data:

# Set the data input and output directories:
population_input_dir = 'C:/Users/mcgr323/projects/tell_valid/tell/tell_raw_data'
mapping_input_dir = 'C:/Users/mcgr323/projects/tell_valid/tell/tell_raw_data'
pop_output_dir = 'C:/Users/mcgr323/projects/tell/ba_hourly_inputs/ba_population'

# Set some processing flags:
start_year = 2015;  # Starting year of time series
end_year = 2019;  # Ending year of time series

**1.3 Meterology data**

Here we use the <im3components> package to load in the WRF meterology data, average WRF meteorology by county and then aggregate them into annual hourly time-series of population-weighted meteorology for each balancing authority (BA). All times are in UTC. Missing values are reported as -9999.

In [ ]:
WRF_input_dir = 'C:/Users/mcgr323/projects/tell_valid/tell/tell_raw_data'
WRF_output_dir = 'C:/Users/mcgr323/projects/tell/BA_hourly_inputs/ba_wrf'

from im3components.wrf_tell.wrf_tell_counties import wrf_to_tell_counties
from im3components.wrf_tell.wrf_tell_balancing_authorities import wrf_to_tell_balancing_authorities



**1.4 Compile hourly load, population and meterology data**

Here we compile all the data processing steps above for hourly load (EIA 930), population (county FIPS) and meteorlogy (WRF) to get a final cleaned up dataset to use as an input to the MLP model. 

In [ ]:
# Compile hourly load, hourly population and hourly WRF data for MLP model
EIA_930_output_dir = 'C:/Users/mcgr323/projects/tell/ba_hourly_inputs/ba_hourly_load'
pop_output_dir = 'C:/Users/mcgr323/projects/tell/ba_hourly_inputs/ba_population'
WRF_output_dir = 'C:/Users/mcgr323/projects/tell/ba_hourly_inputs/ba_wrf'

**2. Model training and prediction**

This step takes the data processed and compiled above and runs a multilayer perceptron (MLP) model to predict future hourly load. 

In [ ]:
import time

data_dir = 'C:/Users/mcgr323/projects/tell/BA_hourly_inputs/compiled_csv_files'
output_dir = 'C:/Users/mcgr323/projects/tell/mlp_model_output'
batch_run = True
target_ba_list = None
generate_plots = True

t0 = time.time()

df = tell.predict(data_dir=data_dir,
                  out_dir=output_dir,
                  batch_run=batch_run,
                  target_ba_list=target_ba_list,
                  generate_plots=generate_plots)

**3. Model forward execution**

This script takes the .csv files produced by the TELL MLP model and distributes the predicted load to the counties that each balancing authority (BA) operates in. The county-level hourly loads are then summed to the state-level and scaled to match the state-level annual loads produced by GCAM-USA. Three sets of output files are generated: county-level hourly loads, state-level hourly loads, and hourly loads for each BA. There is one additional summary output file that includes state-level annual loads from TELL and GCAM-USA as well as the scaling factors.

Please set the directories below to your local machine preferences and run the tell.execute_forward function. 

In [ ]:
# Set the year and GCAM-USA scenario to process:
year_to_process = '2020'
#gcam_usa_scenario = 'scenario_name'

# Set the data input and output directories:
mlp_input_dir = 'C:/Users/mcgr323/projects/tell/mlp_model_output/' + year_to_process + '/'
ba_geolocation_input_dir = 'C:/Users/mcgr323/projects/tell_valid/tell/tell_raw_data'
population_input_dir = 'C:/Users/mcgr323/projects/tell_valid/tell/tell_raw_data'
gcam_usa_input_dir = '/Users/burl878/OneDrive - PNNL/Documents/IMMM/Data/TELL_Input_Data/forward_execution/GCAM_USA_Forcing/Raw_Data/' + gcam_usa_scenario + '/'
data_output_dir = 'C:/Users/mcgr323/projects/tell/forward_model_output/' + year_to_process + '/'

#Run the MLP model forawrd in time and
tell.execute_forward(year_to_process, mlp_input_dir, ba_geolocation_input_dir,
                     population_input_dir, gcam_usa_input_dir, data_output_dir)